In [1]:
!pip install --upgrade typing_extensions
!pip install --upgrade pydantic
!pip install --upgrade gradio
!pip install openai==0.28
!pip install scikit-learn
!pip install --upgrade python
!pip install huggingface_hub

ERROR: Could not find a version that satisfies the requirement python (from versions: none)
ERROR: No matching distribution found for python


In [2]:
import os
import openai
import re
import gradio as gr
import pandas as pd

# Set up OpenAI API key securely
api_key = 'INSERT YORK API KEY HERE'
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key

In [3]:
import gradio as gr
import pandas as pd

model = "gpt-3.5-turbo-instruct"

# Load and preprocess the data
data_url = 'https://raw.githubusercontent.com/sriku2412/GenAI-Digital_Earth_Gardians/main/AI_EarthHack_Dataset_3.csv'
data = pd.read_csv(data_url)
data = data[['summary_problem', 'summary_solution', 'category', 'score', 'rank']]
data.columns = ['problem', 'solution', 'category', 'score', 'rank']
data.dropna(subset=['solution'], inplace=True)
data['solution'] = data['solution'].str.replace('\n', '')
data['solution'] = data['solution'].str.replace('"', '')
data.drop_duplicates(subset=['solution'], inplace=True)
data.sample(10)

,problem,solution,category,score,rank
134,\n\nFinancial Losses: Wasted food represents a...,The initial cost of implementing such a syste...,food wastage,35.0,3
774,NaN,"To kickstart this campaign, we would collabor...",others,34.0,6
464,\n\nOne of the main reasons for these ineffici...,Let's bring this smart innovation to the ener...,Electronic waste and resources utilization,35.0,4
968,\n\nThe improper disposal of e-waste has many...,"By implementing these strategies, companies c...",Electronic waste and resources utilization,35.0,4
129,\nOne of the biggest challenges facing the glo...,Through community collaboration and resource...,food wastage,35.0,3
460,\n\nOne potential solution to this issue is th...,The first compartment of the machine would be ...,plastic waste and usage,34.0,6
888,This results in a significant loss of valuabl...,"Governments, through regulations and incenti...",Electronic waste and resources utilization,36.0,3
477,\n\nPlastic takes hundreds of years to decompo...,"In addition, the DRS could also incentivize m...",plastic waste and usage,36.0,4
663,\n\nFast fashion encourages regular and freque...,"As a result, Blockchain-enabled Fashion as a ...",fashion industry pollution,36.0,3
947,It also encourages skill-sharing and learning...,By reducing the need for people to buy new to...,others,36.0,4


### trial bot 2

In [ ]:
# Function to determine if a text is related to circular economy
def is_related_to_circular_economy(text):
    response = openai.Completion.create(
        model=model,
        prompt=f"Is the following text related to the circular economy? Yes or No.\n\nText: {text}",
        temperature=0,
        max_tokens=60
    )
    return response.choices[0].text.strip().lower() == 'yes'



def evaluate_novelty(solution):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",  # Replace with the specific model for novelty if available
        prompt=f"Evaluate the novelty of the following solution on a scale of 1 to 10. Novelty refers to how original, unusual, or unique the idea is.\n\nSolution: {solution}",
        max_tokens=50
    )
    return extract_score(response.choices[0].text)

def evaluate_utility(solution):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",  # Replace with the specific model for utility if available
        prompt=f"Evaluate the utility of the following solution on a scale of 1 to 10. Utility refers to how useful, valuable, or appropriate the idea is for the given problem or domain.\n\nSolution: {solution}",
        max_tokens=50
    )
    return extract_score(response.choices[0].text)

def evaluate_surprise(solution):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",  # Replace with the specific model for surprise if available
        prompt=f"Evaluate the surprise element of the following solution on a scale of 1 to 10. Surprise refers to how unexpected or surprising the idea is, combining elements of novelty and utility.\n\nSolution: {solution}",
        max_tokens=50
    )
    return extract_score(response.choices[0].text)

def extract_score(text):
    match = re.search(r"\b\d+(\.\d+)?\b", text)
    return float(match.group()) if match else 0

def evaluate_aspect(solution, aspect, prompt_extension):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"{prompt_extension}\n\nSolution: {solution}",
        max_tokens=120  # Increased to capture explanations
    )
    score_match = re.search(r"\b\d+(\.\d+)?\b", response.choices[0].text)
    score = float(score_match.group()) if score_match else 0
    explanation = response.choices[0].text.strip()
    return score, explanation

def evaluate_solution(solution):
    scores = {}
    explanations = {}
    aspects = {
        "novelty": "Evaluate the novelty of the following solution on a scale of 1 to 10. Novelty refers to how original, unusual, or unique the idea is.",
        "utility": "Evaluate the utility of the following solution on a scale of 1 to 10. Utility refers to how useful, valuable, or appropriate the idea is for the given problem or domain.",
        "surprise": "Evaluate the surprise element of the following solution on a scale of 1 to 10. Surprise refers to how unexpected or surprising the idea is, combining elements of novelty and utility."
    }

    for aspect, prompt in aspects.items():
        score, explanation = evaluate_aspect(solution, aspect, prompt)
        scores[aspect] = score
        explanations[aspect] = explanation

    # Calculate the total score
    total_score = sum(scores.values())
    scores['total'] = total_score
    explanations['total'] = "Total score is based on the combined aspects of novelty, utility, and surprise, reflecting the overall creativity and impact of the solution."

    return scores, explanations

def get_good_aspects(solution):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"List the strengths and positive aspects of the following solution: {solution}",
        max_tokens=100
    )
    return response.choices[0].text.strip()

def get_bad_aspects(solution):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"List potential drawbacks or challenges of the following solution: {solution}",
        max_tokens=100
    )
    return response.choices[0].text.strip()

def correct_output(text):
    correction_prompt = f"Please correct any grammar, spelling, numerical, or formatting errors in the following text:\n\n{text}"
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=correction_prompt,
        max_tokens=700  # Adjust as needed
    )
    return response.choices[0].text.strip()

def chatbot_function(user_input):
    try:
        if not is_related_to_circular_economy(user_input):
            return "This solution does not align with circular economy principles.", []

        scores, explanations = evaluate_solution(user_input)
        good_aspects = get_good_aspects(user_input)
        bad_aspects = get_bad_aspects(user_input)
        # Correct total score calculation
        total_score = sum([scores[aspect] for aspect in ["novelty", "utility", "surprise"]])

        # Formatting individual scores in bold and assembling the summary
        summary = "\n".join(f"{aspect.capitalize()}: {scores[aspect]}/10 - {explanations[aspect]}" for aspect in ["novelty", "utility", "surprise"])

        response = f"Total Score: {total_score}/30\n\n" \
                   f"Evaluation Summary:\n{summary}\n\n" \
                   f"Total Score Explanation: {explanations['total']}\n\n" \
                   f"The good: {good_aspects}\n\n" \
                   f"The bad: {bad_aspects}"
        corrected_response = correct_output(response)
        return corrected_response, []

    except Exception as e:
        return f"An error occurred: {e}", []


iface = gr.Interface(
    fn=chatbot_function,
    inputs=gr.Textbox(lines=4, placeholder="Enter your solution"),
    outputs=[gr.Textbox(lines=20), gr.Radio(label="Would you like to know more about any one aspect of the evaluation?")],
    title="Circular Economy Solution Evaluator",
    description="This AI evaluator helps you understand how your solution aligns with circular economy principles and assesses it on novelty, utility, and surprise."
)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://03cec6c04c568c66b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
